In [2]:
SourceTable = 'nw18_youtube_content_owner_basic_a3'
SourceLakehouse = 'NW18_SOCIAL_MEDIA_LH_RAW'
TargetTable = 'youtube_overall_video_stats'
TargetLakehouse = 'NW18_SOCIAL_MEDIA_LH_HISTORY'
LogBatchID = '0'
RunDate = '20241029'

StatementMeta(, 996c9ea5-37ca-4fd6-a350-4638ab9d3c00, 4, Finished, Available, Finished)

In [ ]:
prev_target_count = 0
# target_count_query = f"""
#     SELECT count(*) FROM {TargetLakehouse}.{TargetTable}
# """

# prev_target_count = spark.sql(target_count_query).collect()[0][0]


StatementMeta(, fcda4374-31fc-40dc-8a3b-513293c4f230, 5, Finished, Available, Finished)

In [4]:
from pyspark.sql.functions import col, lit
from datetime import datetime

if RunDate == "CurrentDate" :
    ExecutionDate = str(datetime.now().strftime("%Y%m%d"))
else:
    ExecutionDate = str(RunDate)

artifacts_list = notebookutils.lakehouse.list()

df= spark.createDataFrame(artifacts_list)

prop_df = df.select(
    "displayName",
    col("properties").getItem("abfsPath").alias("abfsPath")
)

source_abfs_path = prop_df.select("abfsPath").filter(col("displayName") == SourceLakehouse ).collect()[0]["abfsPath"]

target_abfs_path = prop_df.select("abfsPath").filter(col("displayName") == TargetLakehouse ).collect()[0]["abfsPath"]

source_file_path = f"{source_abfs_path}/Files/{SourceTable}/{SourceTable}.{ExecutionDate}.parquet"

target_file_path = f"{target_abfs_path}/Files/{TargetTable}"

input_df = spark.read.format("parquet").load(source_file_path)

source_count = input_df.count()

output_df = input_df.withColumn("batch_id", lit(LogBatchID)).withColumn("create_date", lit(datetime.now().strftime("%Y%m%d")))

output_df_without_null = output_df.filter(output_df.video_id.isNotNull())

columns_to_exclude = ['_LATEST_DATE', '_DATA_DATE']

columns_to_consider = [col for col in output_df_without_null.columns if col not in columns_to_exclude]
 
output_unique_df = output_df_without_null.dropDuplicates(columns_to_consider)

# output_unique_df = output_df_without_null.distinct()

target_count = output_unique_df.count()

target_lh_tbl = f"{TargetLakehouse}.{TargetTable}"

if RunDate == "CurrentDate" and TargetTable == "youtube_video_metadata":
    output_unique_df.write.format("delta").mode("overwrite").option("path",target_file_path).saveAsTable(target_lh_tbl)
elif TargetTable != "youtube_video_metadata":
    output_unique_df.write.format("delta").mode("append").option("path",target_file_path).saveAsTable(target_lh_tbl)
else:
    print("No need to load youtube_video_metadata with custom date")

# output_unique_df.write.format("delta").mode("overwrite").option("path",target_file_path).saveAsTable(target_lh_tbl)

StatementMeta(, 996c9ea5-37ca-4fd6-a350-4638ab9d3c00, 6, Finished, Available, Finished)

In [16]:
import json

result_dict = {
    "target_prev_count": prev_target_count,
    "source_count": source_count,
    "target_count": target_count,
    "log_batch_id": LogBatchID
}

result_json = json.dumps(result_dict, indent=4)

notebookutils.notebook.exit(result_json)

StatementMeta(, a9e02d43-c994-4101-ac83-a0ebde36661e, 18, Finished, Available, Finished)

ExitValue: {
    "target_prev_count": 59997296,
    "source_count": 1327538,
    "target_count": 1327538,
    "log_batch_id": "0"
}